# MeCab & CaboCha

## CONTENTS

* [MeCab](#MeCabによる形態素解析)
* [CaboCha](#CaboChaによる係り受け解析)

## MeCabによる形態素解析

### 形態素解析とは

文法的な情報の注記の無い自然言語の文から、対象言語の文法や、辞書と呼ばれる単語の品詞等の情報にもとづき、  
形態素(おおまかにいえば、言語で意味を持つ最小単位)に分割し、それぞれの形態素の品詞等を判別する作業。


<img src="./figures/explain-morpheme.png" width="70%" height="70%"/>


### [MeCab(和布蕪)](https://taku910.github.io/mecab/)

オープンソース形態素解析エンジン

In [1]:
import MeCab

#### とりあえず解析

`MeCab.Tagger` クラスインスタンスを生成して、 `parse` というメソッドを用いて、形態素解析の結果を文字列として得ることができる。  
MeCabでは、一行一文を前提に解析が行われる。

In [2]:
tagger = MeCab.Tagger()
sentence = 'いま働いている会社は最悪な会社だと思う'
parse = tagger.parse(sentence)

In [3]:
print(parse)

いま	名詞,副詞可能,*,*,*,*,いま,イマ,イマ
働い	動詞,自立,*,*,五段・カ行イ音便,連用タ接続,働く,ハタライ,ハタライ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
会社	名詞,一般,*,*,*,*,会社,カイシャ,カイシャ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
最悪	名詞,形容動詞語幹,*,*,*,*,最悪,サイアク,サイアク
な	助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ
会社	名詞,一般,*,*,*,*,会社,カイシャ,カイシャ
だ	助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ
と	助詞,格助詞,引用,*,*,*,と,ト,ト
思う	動詞,自立,*,*,五段・ワ行促音便,基本形,思う,オモウ,オモウ
EOS



In [4]:
type(parse)

str

#### 各形態素の詳細情報の取得

`MeCab.Tagger` クラスインスタンスの `parseToNode` というメソッドにより、  
<u>**文頭**</u>という特別な形態素が、 `MeCab.Node` クラスインスタンスとして取得できる。

In [5]:
tagger = MeCab.Tagger()
tagger.parse('') # MeCabのバグ対応
node = tagger.parseToNode(sentence)

In [6]:
type(node)

MeCab.Node

MeCab.Node クラスインスタンスは、双方向リストとして表現されていて、 `next` 、 `prev` という属性がある。  
それぞれ、次の形態素、前の形態素を `MeCab.Node` クラスインスタンスとして取得できる。  
また、形態素の文字列情報と素性情報は、`MeCab.Node` クラスインスタンスの `surface` 、 `feature` という属性により取得できる。

In [7]:
print(node.next.surface)

いま


ここで、 `parseToNode` で取得した初めの `node` は、 BOS(beginning of sentence) として空文字列('') として取得されるので、注意。  
行末の `node` も、EOS(end of sentence)として空文字列('') として取得される。

In [8]:
node = node.next.next

In [9]:
print(f'1:{node.prev.surface} 2:{node.surface} 3:{node.next.surface}')

1:いま 2:働い 3:て


一文の全形態素の文字列情報と素性情報を取得したい場合は、以下の様にするとよい。

In [10]:
node = tagger.parseToNode(sentence)

surface_list = list()
feature_list = list()

while node:
    surface_list.append(node.surface)
    feature_list.append(node.feature)
    node = node.next

In [11]:
surface_list

['', 'いま', '働い', 'て', 'いる', '会社', 'は', '最悪', 'な', '会社', 'だ', 'と', '思う', '']

In [12]:
feature_list

['BOS/EOS,*,*,*,*,*,*,*,*',
 '名詞,副詞可能,*,*,*,*,いま,イマ,イマ',
 '動詞,自立,*,*,五段・カ行イ音便,連用タ接続,働く,ハタライ,ハタライ',
 '助詞,接続助詞,*,*,*,*,て,テ,テ',
 '動詞,非自立,*,*,一段,基本形,いる,イル,イル',
 '名詞,一般,*,*,*,*,会社,カイシャ,カイシャ',
 '助詞,係助詞,*,*,*,*,は,ハ,ワ',
 '名詞,形容動詞語幹,*,*,*,*,最悪,サイアク,サイアク',
 '助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ',
 '名詞,一般,*,*,*,*,会社,カイシャ,カイシャ',
 '助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ',
 '助詞,格助詞,引用,*,*,*,と,ト,ト',
 '動詞,自立,*,*,五段・ワ行促音便,基本形,思う,オモウ,オモウ',
 'BOS/EOS,*,*,*,*,*,*,*,*']

形態素の品詞を抜き出したければ、以下のようにする。

In [13]:
[ feature.split(',')[0] for feature in feature_list]

['BOS/EOS',
 '名詞',
 '動詞',
 '助詞',
 '動詞',
 '名詞',
 '助詞',
 '名詞',
 '助動詞',
 '名詞',
 '助動詞',
 '助詞',
 '動詞',
 'BOS/EOS']

#### `MeCab.Tagger` 

##### **クラスのコンストラクタの引数**

全ての引数は、公式の[こちら](https://taku910.github.io/mecab/mecab.html)を参照のこと。  
基本的には、コマンドライン実行形式の引数に同じ。  
以下には、よく使うものを列挙

* `-r`, `--rcfile=FILE`  
mecabの設定ファイルとして使用するものを指定する
* `-d`, `--dicdir=DIR`  
システムディクショナリのディレクトリを指定する。
    ```
    tagger = MeCab.Tagger('-d /usr/local/lib/mecab/dic/ipadic')
    ```
* `-u`, `--userdic=FILE`  
ユーザーディクショナリのファイルを指定する。
    ```
    tagger = MeCab.Tagger('-u ./usr-mecab-ipadic-neologd.dic')
    ```

##### **method/attribute**

MeCab.Tagger クラスのは[こちら](https://taku910.github.io/mecab/bindings.html)を参照のこと

## CaboChaによる係り受け解析

### 係り受け解析とは

文法的な情報の注記の無い自然言語の文から、対象言語の文法や、辞書と呼ばれる単語の品詞等の情報にもとづき、  
文節 (日本語の文法においては、文を細かく区切った際に不自然にならない最小単位。単語よりは大きい)に分割し、  
それぞれの修飾・被修飾関係を判別する作業。

<img src="./figures/explain-modification.png" width="45%" height="45%"/>

### [CaboCha(南瓜)](http://taku910.github.io/cabocha/)

CaboChaは、日本語係り受け解析器

In [14]:
import CaboCha

#### とりあえず係り受け解析

`CaboCha.Parser` クラスインスタンスを生成して、`parse` というメソッドを用いて、係り受け解析の結果を `CaboCha.Tree` クラスインスタンスとして得ることができる。  
`CaboCha.Tree` クラスインスタンスの `toString` というメソッドを用いて、解析結果を文字列として取得できる。  

In [15]:
parser = CaboCha.Parser()
sentence = 'いま働いている会社は最悪な会社だと思う'
tree = parser.parse(sentence)
result = tree.toString(CaboCha.FORMAT_TREE)

In [16]:
print(result)

      いま-D        
  働いている-D      
        会社は---D  
          最悪な-D  
          会社だと-D
                思う
EOS



In [17]:
type(result)

str

#### 各形態素の詳細情報の取得

形態素のデータを取得するには、`parse` メソッドで得られた `CaboCha.Tree` クラスインスタンスの、 `token()` メソッドを用いれば良い。  
このメソッドの引数は、int型で、何番目の<u>**形態素**</u>を取得するかの値をとる。  
解析の結果得られた、全形態素数は、`CaboCha.Tree` クラスインスタンスの、 `token_size` メソッドにより取得できる。  
形態素のデータは、 `CaboCha.Token` クラスインスタンスとして、以下のように取得できる。  

In [18]:
token_list = [ tree.token(n) for n in range(tree.token_size())]

In [19]:
type(token_list[0])

CaboCha.Token

`CaboCha.Token` クラスインスタンスの `surface` 、 `feature` という属性により形態素の文字列情報と素性情報を取得できる。

In [20]:
surface_list = [ token.surface for token in token_list]
surface_list

['いま', '働い', 'て', 'いる', '会社', 'は', '最悪', 'な', '会社', 'だ', 'と', '思う']

In [21]:
feature_list = [ token.feature for token in token_list]
feature_list

['名詞,副詞可能,*,*,*,*,いま,イマ,イマ',
 '動詞,自立,*,*,五段・カ行イ音便,連用タ接続,働く,ハタライ,ハタライ',
 '助詞,接続助詞,*,*,*,*,て,テ,テ',
 '動詞,非自立,*,*,一段,基本形,いる,イル,イル',
 '名詞,一般,*,*,*,*,会社,カイシャ,カイシャ',
 '助詞,係助詞,*,*,*,*,は,ハ,ワ',
 '名詞,形容動詞語幹,*,*,*,*,最悪,サイアク,サイアク',
 '助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ',
 '名詞,一般,*,*,*,*,会社,カイシャ,カイシャ',
 '助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ',
 '助詞,格助詞,引用,*,*,*,と,ト,ト',
 '動詞,自立,*,*,五段・ワ行促音便,基本形,思う,オモウ,オモウ']

結果をみるとわかるように、CaboCha での係り受け解析には、形態素解析が実施されている。このとき MeCab が使用されている。

#### 文節の詳細情報の取得

係り受け解析の末得られた文節のデータを取得するには、`parse` メソッドで得られた `CaboCha.Tree` クラスインスタンスの、`chunk` メソッドを用いれば良い。  
このメソッドの引数は、int型で、何番目の<u>**文節**</u>を取得するかの値をとる。  
解析の結果得られた、全文節数は、`CaboCha.Tree` クラスインスタンスの、`chunk_size` メソッドにより取得できる。  
文節のデータは、`CaboCha.Chunk` クラスインスタンスとして、以下のように取得できる。

In [22]:
chunk_list = [ tree.chunk(n) for n in range(tree.chunk_size()) ]

In [23]:
type(chunk_list[0])

CaboCha.Chunk

とある文節がどの文節にかかっているかは、 `link` という属性により取得できる。

In [24]:
chunk_list[0].link

1

この場合、一番目の文節は、二番目の文節にかかっている事になる。どの文節にもかからない場合は、 0 が返る。

また、各文節の文字列情報と素性情報を取得するメソッドは、`CaboCha.Chunk` クラスインスタンスには用意されていない。  
そのため、形態素のデータと組み合わせて取得することになる。

各文節に含まれる形態素の数は、 `CaboCha.Chunk` クラスインスタンスの `token_size` という属性により取得できる。  
また、ひとつの文節中の一番初めの形態素の全形態素中でのインデックスを `token_pos` という属性により取得できる。

In [25]:
chunk_list[2].token_size

2

In [26]:
chunk_list[2].token_pos

4

これらの値を用いて、以下のように文節を取得できる。

In [27]:
paragraph = [ str().join(surface_list[chunk.token_pos:chunk.token_pos+chunk.token_size]) for chunk in chunk_list ]
paragraph

['いま', '働いている', '会社は', '最悪な', '会社だと', '思う']

#### 係り受けの関係の取得

係り受けの関係をまとめる方法の一例として以下の方法を挙げる。

In [28]:
chunk_link_list = np.array([ chunk.link for chunk in chunk_list ])
paragraph = np.array(paragraph)
dependency = dict(zip(paragraph[chunk_link_list >= 0],paragraph[chunk_link_list[chunk_link_list >= 0]]))
for active, passive in dependency.items():
    print(f'{active} -> {passive}')

いま -> 働いている
働いている -> 会社は
会社は -> 会社だと
最悪な -> 会社だと
会社だと -> 思う


#### `CaboCha.Parser`

##### **クラスのコンストラクタの引数**

全ての引数は、公式の[こちら](http://taku910.github.io/cabocha/)の**コマンドラインオプション**の項を参照のこと。  
よく使いそうなものを以下に挙げる。  

* `-d`, `--mecab-dicdir=DIR`  
システムディクショナリのディレクトリを指定する。
```
tagger = CaboCha.Parser('-d /usr/local/lib/mecab/dic/ipadic')
```